In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from mlp import model
import tensorflow as tf

In [2]:
train_label = pd.read_csv('/Users/hongseoklee/VSCodeWorkspace/AIs/NCSoft/final_data/train/train_label.csv')
print(train_label)


train_activity = pd.read_csv('/Users/hongseoklee/VSCodeWorkspace/AIs/NCSoft/final_data/train/train_activity.csv')
print(train_activity)


df = pd.merge(left = train_label , right = train_activity, how = "inner", on = "acc_id")
print(df)

                                                  acc_id     label
0      b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...      week
1      ed500c4957956b3e99dc3985666850b582f812405eefb6...      week
2      acc6afa23a6bf15e18151e4794c7789225ef9d682f473c...      week
3      34095a3c9a2937ced3ea3fd75e22ce177dc5879d2a53f7...      week
4      26f3db6e8817a93c4ceda9a16f0832945e43d950b95882...      week
...                                                  ...       ...
99995  da6d33b03968d8e35821f6eb88ad22e12e37aa8867084e...  retained
99996  676c944f4b6ae63818b3cad824a61233690f16a2275d5d...  retained
99997  695e1f28e234fc4cc53085e332fa7a76d7895ca4cc745b...  retained
99998  0c87fabaad5542e533f958a1d6fd739993b94e95e00989...  retained
99999  47ff575cb94019df5695c5d81ec285b0d801607b2a8697...  retained

[100000 rows x 2 columns]
        wk                                             acc_id  cnt_dt  \
0        7  3dc6f2875dc6e6f35b9e2bdb25b391a8003386ff23becd...       4   
1        8  3dc6f2875dc

In [3]:
categorical_features = ["acc_id"]
# for문을 사용하여 Label Encoding 적용

# 원하는 라벨 순서로 fit 수행

label_encoder = LabelEncoder()

# 원하는 라벨 순서 정의
label_mapping = {'week':0, 'month':1, '2month':2, 'retained':3}

# 데이터프레임 열에 매핑 적용
df['label'] = df['label'].map(label_mapping)

print(df)

for column in categorical_features:
    df[column] = label_encoder.fit_transform(df[column])

                                                   acc_id  label  wk  cnt_dt  \
0       b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...      0   8       3   
1       ed500c4957956b3e99dc3985666850b582f812405eefb6...      0   5       7   
2       ed500c4957956b3e99dc3985666850b582f812405eefb6...      0   6       3   
3       ed500c4957956b3e99dc3985666850b582f812405eefb6...      0   7       6   
4       ed500c4957956b3e99dc3985666850b582f812405eefb6...      0   8       7   
...                                                   ...    ...  ..     ...   
440318  47ff575cb94019df5695c5d81ec285b0d801607b2a8697...      3   4       3   
440319  47ff575cb94019df5695c5d81ec285b0d801607b2a8697...      3   5       4   
440320  47ff575cb94019df5695c5d81ec285b0d801607b2a8697...      3   6       5   
440321  47ff575cb94019df5695c5d81ec285b0d801607b2a8697...      3   7       7   
440322  47ff575cb94019df5695c5d81ec285b0d801607b2a8697...      3   8       7   

        play_time   npc_exp  npc_hongmu

In [4]:
train_x = df.drop(labels = ["label", "acc_id"], axis = 1)
train_y = df["label"]

columns = train_x.columns

print(train_x)

print(train_x.shape)

        wk  cnt_dt  play_time   npc_exp  npc_hongmun  quest_exp  \
0        8       3   1.519815  2.910685    -0.268110   2.852241   
1        5       7   4.698532  9.147854    -0.138458   8.584935   
2        6       3   0.632257  0.213292    -0.290501  -0.022940   
3        7       6   1.218180  1.787315    -0.159228   1.727487   
4        8       7   0.521166 -0.231874    -0.030257  -0.250423   
...     ..     ...        ...       ...          ...        ...   
440318   4       3  -0.486890 -0.231874    -0.251545  -0.250423   
440319   5       4  -0.035085 -0.231874    -0.004726  -0.250423   
440320   6       5   0.445482 -0.231874    -0.054757  -0.250423   
440321   7       7   0.832217 -0.231874     0.484635  -0.250423   
440322   8       7   1.132428 -0.231874     0.461161  -0.250423   

        quest_hongmun  item_hongmun  game_combat_time  get_money  ...  \
0            2.332910     -0.306354          0.727017  -0.023484  ...   
1            5.995250     -0.306354          2.98

In [5]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # 정확한 예측의 개수 계산
        result = list(map(int, model.predict(val_x)))
        y_pred = []
        for y in result:
            if y < 0:
                y_pred.append(0)
            elif y > 3:
                y_pred.append(3)
            else:
                y_pred.append(y)
        
        y_true = val_y
        correct_predictions = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    
        # 전체 예측의 개수
        total_predictions = len(y_true)
    
        # 정확도 계산
        accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0
            
        # 에폭이 끝날 때마다 특정 값 출력 (예: 손실 함수 값)
        print(f"At the end of epoch {epoch + 1}, accuracy: {accuracy}")
        print(y_pred[:10], y_true[:10])


In [ ]:
selected_features = ['item_hongmun', 'party_chat', 'cnt_dt', 'npc_hongmun', 'whisper_chat', 'wk', 'cnt_use_buffitem', 'get_money', 'quest_hongmun', 'guild_chat', 'game_combat_time', 'play_time']
train_x = train_x[selected_features]

#검증 데이터셋 추출
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# 콜백 생성
custom_callback = CustomCallback()

model.fit(train_x, train_y, epochs=30, batch_size=32, callbacks=[custom_callback])

Epoch 1/30
2753/2753 [==============================] - 1s 276us/step
At the end of epoch 1, accuracy: 0.38038948503945946
[2, 1, 2, 1, 2, 2, 1, 1, 2, 2] 288041    3
198927    2
369752    3
112785    1
378827    3
363017    3
357116    3
58393     0
296580    3
197749    2
Name: label, dtype: int64
11009/11009 [==============================] - 10s 904us/step - loss: 0.8008 - accuracy: 0.2574
Epoch 2/30
2753/2753 [==============================] - 1s 265us/stepo
At the end of epoch 2, accuracy: 0.29299948901379663
[2, 1, 1, 0, 2, 2, 1, 1, 2, 1] 288041    3
198927    2
369752    3
112785    1
378827    3
363017    3
357116    3
58393     0
296580    3
197749    2
Name: label, dtype: int64
11009/11009 [==============================] - 10s 867us/step - loss: 0.7064 - accuracy: 0.2705
Epoch 3/30
2753/2753 [==============================] - 1s 267us/stepo
At the end of epoch 3, accuracy: 0.3198432975642991
[2, 1, 1, 1, 2, 2, 1, 1, 2, 1] 288041    3
198927    2
369752    3
112785    1
37882

In [ ]:
print(model.predict(train_x[:10]), train_y[:10])